# Data Cleaning & Preparation
This notebook loads both Western (e.g., Sephora) and Korean (Olive Young) skincare data.

Goals:
- Merge and clean both datasets
- Standardize key columns (product type, ingredients)
- Drop outliers and invalid entries
- Prepare for visual EDA

Final output: a cleaned dataframe for analysis.

In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine, text
pd.set_option('display.max_columns', None)

In [2]:
#INITIAL CLEANING

#Understanding starting datasets
olive = pd.read_csv('/Users/sarahhyun/skincare/data/olive/oliveyoung_all_scraped_batches.csv') #korean skincare dataset
olive_spf = pd.read_csv('/Users/sarahhyun/skincare/data/olive/oliveyoung_sunscreen_scrapedv2.csv') #separate dataset for sunscreen
sephora = pd.read_csv('/Users/sarahhyun/skincare/data/sephora/product_info.csv') #sephora skincare dataset
print(olive.head())
print(olive.dtypes)
print(sephora.head())
print(sephora.dtypes)


                                            prdtName       Brand  \
0                  AESTURA Atobarrier 365 Cream 80mL     AESTURA   
1         Dr.G Red Blemish Clear Soothing Cream 70ml        Dr.G   
2  d'Alba White Truffle First Spray Serum 100mL D...      d'Alba   
3         SKIN1004 Madagascar Centella Ampoule 100ml    SKIN1004   
4                   Dr. Althea 345 Relief Cream 50ml  Dr. Althea   

                                                 url  rating-info     price  \
0  https://global.oliveyoung.com/product/detail?p...          4.8  US$39.60   
1  https://global.oliveyoung.com/product/detail?p...          4.8  US$37.80   
2  https://global.oliveyoung.com/product/detail?p...          4.8  US$41.20   
3  https://global.oliveyoung.com/product/detail?p...          4.9  US$18.70   
4  https://global.oliveyoung.com/product/detail?p...          4.8  US$28.72   

                                         ingredients           skin_type  \
0  Water, Butylene Glycol, Glycerine, Bu

In [3]:
#SEPHORA DATA CLEANING PART 1

# print(sephora['primary_category'].value_counts())

#only keep skincare products from primary category
skincare = sephora[sephora['primary_category'] == 'Skincare']
skincare = skincare.reset_index(drop=True)
skincare.head()

#reviewing secondary categories
# print(skincare['secondary_category'].value_counts())

#confirming we don't care about the following secondary categories
skincare[
        (skincare['secondary_category'] == 'Self Tanners') |
        (skincare['secondary_category'] == 'Wellness') |
        (skincare['secondary_category'] == 'Value & Gift Sets') |
        (skincare['secondary_category'] == 'Mini Size') |
        (skincare['secondary_category'] == 'High Tech Tools') |
        (skincare['secondary_category'] == 'Shop by Concern')
         #(skincare['secondary_category'] == 'Lip Balms & Treatments')  #determined these rows should be included
     ]

#removing secondary categories
skincare = skincare[(skincare['secondary_category'] != 'Self Tanners') &
        (skincare['secondary_category'] != 'Wellness') &
        (skincare['secondary_category'] != 'Value & Gift Sets') &
        (skincare['secondary_category'] != 'Mini Size') &
        (skincare['secondary_category'] != 'High Tech Tools') &
        (skincare['secondary_category'] != 'Shop by Concern')].reset_index(drop = True)

#reviewing categories in tertiary
# print(skincare['tertiary_category'].value_counts())

#confirming we don't care about the following tertiary categories
skincare[
        (skincare['tertiary_category'] == 'Decollete & Neck Creams') |
        # (skincare['tertiary_category'] == 'Eye Masks') |  #determined these rows should be included
        (skincare['tertiary_category'] == 'Body Sunscreen') |
        (skincare['tertiary_category'] == 'Makeup Removers') |
        (skincare['tertiary_category'] == 'Face Wipes') |
        (skincare['tertiary_category'] == 'Blotting Papers') |
        (skincare['tertiary_category'] == 'BB & CC Creams')
     ]

#removing tertiary categories
skincare = skincare[(skincare['tertiary_category'] != 'Decollete & Neck Creams') &
        (skincare['tertiary_category'] != 'Body Sunscreen') &
        (skincare['tertiary_category'] != 'Makeup Removers') &
        (skincare['tertiary_category'] != 'Face Wipes') &
        (skincare['tertiary_category'] != 'Blotting Papers') &
        (skincare['tertiary_category'] != 'BB & CC Creams')].reset_index(drop = True)

# print("\n Final review of categories")
# print(skincare['primary_category'].value_counts())
# print(skincare['secondary_category'].value_counts())
# print(skincare['tertiary_category'].value_counts())

#replacing na values in tertiary with secondary category values and replace blemish & acne
skincare['tertiary_category'] = skincare['tertiary_category'].fillna(skincare['secondary_category'])
skincare['tertiary_category'].isna().sum()
skincare['tertiary_category']=skincare['tertiary_category'].replace('Blemish & Acne Treatments', 'Treatments')

len(skincare) #final number of rows: 1855, originally 8429

1855

In [4]:
# SEPHORA DATA CLEANING PART 2: RENAMING COLUMNS & FINAL CLEANUP
columns = [
    'product_name',
    'brand_name',
    'rating',
    'reviews',
    'size',
    'price_usd',
    'tertiary_category',
    'ingredients'
]

products_sephora = skincare[columns]
products_sephora.head(20)

products_sephora = products_sephora.rename(columns = {'tertiary_category':'type'})
products_sephora.head()
products_sephora.isnull().sum()

#make sure type values are consistent 
products_sephora['type'].value_counts() #we want to combine face wash & cleansers with cleaners and night creams with moisturizers
products_sephora['type'] = products_sephora['type'].replace({
    'Face Wash & Cleansers': 'Cleansers', 
    'Night Creams': 'Moisturizers', 
    'Mists & Essences': 'Toners', 
    'Lip Balms & Treatments': 'Moisturizers', 
    'Eye Creams & Treatments': 'Moisturizers',
    'Eye Masks': 'Treatments',
    'Face Serums': 'Serums',
    'Face Masks': 'Treatments',
    'Sheet Masks': 'Treatments',
    'Face Sunscreen': 'Sunscreen'
})
print(products_sephora.groupby('type')['price_usd'].describe())
products_sephora.isnull().sum()


products_sephora.to_csv("/Users/sarahhyun/skincare/data/sephora/sephora_cleaned.csv", index=False)

              count       mean        std    min    25%   50%   75%    max
type                                                                      
Cleansers     220.0  34.663318  17.558698   5.99  25.00  33.0  39.0   95.0
Exfoliators    41.0  48.963415  25.754221  14.00  34.00  39.0  64.0  135.0
Face Oils      66.0  62.249697  39.692013   7.50  39.25  60.0  73.5  270.0
Facial Peels   50.0  53.749800  30.541360   9.00  35.00  47.5  72.0  125.0
Moisturizers  637.0  69.612904  58.323935   5.00  37.00  56.0  75.0  400.0
Serums        379.0  73.262612  63.364302   5.00  35.00  65.0  89.0  425.0
Sunscreen      98.0  44.591735  37.612548  14.99  30.00  36.0  42.0  270.0
Toners        147.0  47.982857  47.127458   6.00  24.00  35.0  49.0  315.0
Treatments    217.0  44.779908  40.792645   3.00  17.00  39.0  55.0  300.0


product_name     0
brand_name       0
rating          36
reviews         36
size            59
price_usd        0
type             0
ingredients     30
dtype: int64

In [5]:
#OLIVE YOUNG DATA CLEANING PART 1
#removing lines where it's a set, refill, pack, 2 pcs, etc

pattern = r"""(?ix)                  # i: case-insensitive, x: allow comments/whitespace
(
    \b(set|duo|refill|bundle|kit|twin\s*pack|value\s*set|gift\s*set)\b
  | \b(x\s*\d+)\b                    # e.g., x2, x3
  | \b\d+\s*(ea|pcs?|pack)s?\b       # e.g., 2ea, 3pcs, 2pack
  | \d+\s*(ml|g)\s*\+\s*\d+\s*(ml|g) # e.g., 30ml+30ml, 100g+50g
)
"""


mask_setlike = olive["prdtName"].str.contains(pattern, regex=True, na=False)

# review what will be dropped (spot-check)
print("Rows flagged as set/duo/refill/pack:", mask_setlike.sum())
print(olive.loc[mask_setlike, "prdtName"].head(15).to_list())

# keep only non-set items
olive_clean = olive.loc[~mask_setlike].copy()

# olive_clean.to_csv("olive_products_no_sets.csv", index=False)
# olive.loc[mask_setlike].to_csv("oliveyoung_all_scraped_batches.csv", index=False)

Rows flagged as set/duo/refill/pack: 0
[]


/var/folders/gp/x0x03fs11gjckcjvh3wkw6jw0000gn/T/ipykernel_78336/833375714.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_setlike = olive["prdtName"].str.contains(pattern, regex=True, na=False)


In [6]:
#OLIVE YOUNG DATA CLEANING PART 2
#updating column names and data types
products_olive= pd.read_csv('/Users/sarahhyun/skincare/data/olive/oliveyoung_all_scraped_batches.csv')
products_olive.dtypes
products_olive.isnull().sum()

column_rename={
    'prdtName': 'product_name',
    'Brand': 'brand_name',
    'rating-info': 'rating',
    'amount': 'size',
    'price': 'price_usd',
    'product_type': 'type'}
products_olive = products_olive.rename(columns = column_rename)
products_olive.head()

#updating price column to remove "US$" and make it to clean float
products_olive['price_usd'] = (
    products_olive['price_usd'].astype(str).str.replace('US$', '', regex=False).str.replace(',', '', regex=False).astype(float)
    )
products_olive.isnull().sum()
products_olive[['price_usd','rating']].describe()


#clean up type options 
products_olive = products_olive[products_olive['type'] != 'Other'].reset_index(drop=True)
drop_rows = ['Men Shave', 'Shampoo','Makeup Remover']
products_olive = products_olive[~products_olive['type'].isin(drop_rows)].reset_index(drop=True)

#rename type values so it's more consistent
products_olive['type'] = products_olive['type'].replace({
    'Cream': 'Moisturizers',
    'Ampoule':'Toners',
    'Toner': 'Toners',
    'Cleansing Foam': 'Cleansers',
    'Cleansing Oil': 'Cleansers',
    'Essence': 'Toners',
    'Cleansing Balm': 'Cleansers',
    'Lotion':'Moisturizers',
    'Cleansing Gel': 'Cleansers',
    'Gel Cream': 'Moisturizers',
    'Emulsion': 'Toners',
    'Exfoliating Toner': 'Exfoliators',
    'Mist': 'Toners',
    'Mask': 'Treatments',
    'Eye Cream': 'Moisturizers',
    'Spot Treatment': 'Treatments',
    'Oil Cleanser': 'Cleansers',
    'Wash-Off Mask':'Treatments',
    'Toner Pad': 'Toners',
    'Sheet Mask': 'Treatments',
    'Pimple Patch': 'Treatments',
    'Face Sunscreen': 'Sunscreen',
    'Moisturizer': 'Moisturizers',
    'Cleanser': 'Cleansers',
    'Serum': 'Serums',
    'Treatment':'Treatments'
})
products_olive.type.value_counts()

#Face sunscreens were not included in original scrape. performed another data scraping. combining both csv files together manually 


print(products_olive.groupby('type')['price_usd'].describe())

products_olive['price_usd'].isna().groupby(products_olive['type']).sum()

#Save cleaned up oliveyoung data
products_olive.isnull().sum()

products_olive.to_csv("/Users/sarahhyun/skincare/data/olive/olive_cleaned.csv", index=False)

              count       mean        std    min      25%     50%      75%  \
type                                                                         
Cleansers     365.0  18.454685   7.877191   2.45  13.3000  18.000  22.0200   
Exfoliators    21.0  19.971429   6.478005  10.34  17.2500  19.000  21.0000   
Face Oils       6.0  24.645000   7.861539  12.85  20.1650  25.705  29.5650   
Facial Peels   15.0  14.958667   5.339486   9.26  11.6500  14.130  16.2700   
Moisturizers  403.0  27.476005  11.838941   6.47  20.1250  25.600  32.8000   
Serums        245.0  27.203388  10.837498   6.00  19.9100  26.100  31.8100   
Sunscreen       2.0  31.750000   7.990307  26.10  28.9250  31.750  34.5750   
Toners        402.0  25.564478  10.933889   8.40  18.5875  23.100  29.6000   
Treatments    120.0  14.991333   8.733174   3.62   8.8000  12.600  18.2175   

                 max  
type                  
Cleansers      68.97  
Exfoliators    44.65  
Face Oils      34.44  
Facial Peels   31.21  
Moi

product_name      0
brand_name        0
url               0
rating          400
price_usd       553
ingredients      53
skin_type       310
size            314
type              0
dtype: int64

In [8]:
#MERGING SEPHORA AND OLIVE DATA


#double checking ingredient column datatype before merging
sephora = pd.read_csv('/Users/sarahhyun/skincare/data/sephora/sephora_cleaned.csv')
olive = pd.read_csv('/Users/sarahhyun/skincare/data/olive/olive_cleaned.csv')
sephora.ingredients.dtypes
sephora.ingredients.apply(type).value_counts() #1825 string values and 30 float
olive.ingredients.dtypes
olive.ingredients.apply(type).value_counts() #2079 string values and 53 float
sephora.ingredients = sephora.ingredients.apply(lambda x: x[2:-3] if isinstance(x, str) else x)

#normalize columns on both datasets
CORE_COLS = ["product_name","brand_name","rating", "price_usd", "type", "ingredients", "product_url"]

def to_numeric(s):
    return pd.to_numeric(s, errors="coerce")

def normalize_olive(df):
    df = df.copy()
    # drop the columns you don't want from Olive
    df = df.drop(columns=["skin_type","size"], errors="ignore")
    for c in CORE_COLS:
        if c not in df.columns:
            df[c] = pd.NA
    # dtypes
    df["rating"]    = to_numeric(df["rating"])
    df["price_usd"] = to_numeric(df["price_usd"])
    # tidy url (ensure absolute)
    if "product_url" in df.columns:
        df["product_url"] = df["product_url"].apply(
            lambda u: (u if (pd.notna(u) and str(u).startswith("http"))
                       else (urljoin("https://global.oliveyoung.com", str(u)) if pd.notna(u) else pd.NA))
        )
    return df[CORE_COLS]

def normalize_sephora(df):
    df = df.copy()
    df = df.drop(columns=["reviews", "size"], errors="ignore")
    # ensure expected columns exist
    for c in CORE_COLS:
        if c not in df.columns:
            df[c] = pd.NA
    # dtypes
    df["rating"]    = to_numeric(df["rating"])
    df["price_usd"] = to_numeric(df["price_usd"])
    return df[CORE_COLS]

seph_n  = normalize_sephora(sephora)
olive_n = normalize_olive(olive)

seph_n["source"]  = "western_sephora"
olive_n["source"] = "kbeauty_olive"

#combine rows
skincare_full = pd.concat([seph_n, olive_n], ignore_index=True)
skincare_full.head()

#download as CSV
skincare_full.to_csv("/Users/sarahhyun/skincare/data/skincare_full.csv", index=False)
#manually added additional sunscreen data scraped separately from olive young

,product_name,brand_name,rating,price_usd,type,ingredients,product_url,source
0,GENIUS Sleeping Collagen Moisturizer,Algenist,4.5413,98.0,Moisturizers,"Collagen (Vegan)*, Water (Aqua, Eau), Ethylhex...",NaN,western_sephora
1,GENIUS Liquid Collagen Serum,Algenist,4.0259,115.0,Face Serums,"Collagen (Vegan)*, Water (Aqua, Eau), Propaned...",NaN,western_sephora
2,Triple Algae Eye Renewal Balm Eye Cream,Algenist,4.5306,68.0,Eye Creams & Treatments,"Aqua (Water/Eau), Stearic Acid, Isopropyl Isos...",NaN,western_sephora
3,GENIUS Liquid Collagen Lip Treatment,Algenist,3.8721,29.0,Lip Balms & Treatments,"Collagen (Vegan)*, Water (Aqua, Eau), Glycerin...",NaN,western_sephora
4,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,Algenist,4.4134,28.0,Face Sunscreen,"Octinoxate 7.5%, Titanium Dioxide 2%, Zinc Oxi...",NaN,western_sephora
